In [46]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [47]:
img_size = (48, 48)
batch_size = 32

# Ruta actualizada
dataDir = "dataset/train"

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataDir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size
)

class_names = original_train_ds.class_names
# Normalización
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = original_train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 28709 files belonging to 7 classes.
Using 5741 files for validation.


## Entrenamiento: 

In [48]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# 🔄 Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x,training=True), y))

# Modelo mejorado
model = models.Sequential([
    keras.layers.Input(shape=(48, 48, 1)),

    keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(class_names), activation='softmax')
])

# ⚙️ Compilación
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ⏹️ EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)


# ⚖️ Class weights
y_train = []
for _, labels in train_ds.unbatch():
    y_train.append(labels.numpy())
y_train = np.array(y_train)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights))


# 🏋️ Entrenamiento
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[early_stop],
    class_weight=class_weights_dict
)


Epoch 1/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 59s 80ms/step - accuracy: 0.1517 - loss: 3.1684 - val_accuracy: 0.0674 - val_loss: 2.4272
Epoch 2/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 57s 79ms/step - accuracy: 0.1677 - loss: 2.0539 - val_accuracy: 0.1742 - val_loss: 1.9688
Epoch 3/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 57s 80ms/step - accuracy: 0.1748 - loss: 1.9494 - val_accuracy: 0.1552 - val_loss: 1.9448
Epoch 4/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 61s 85ms/step - accuracy: 0.1791 - loss: 1.9186 - val_accuracy: 0.2163 - val_loss: 1.8875
Epoch 5/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 59s 83ms/step - accuracy: 0.2157 - loss: 1.8635 - val_accuracy: 0.2662 - val_loss: 1.8155
Epoch 6/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 60s 84ms/step - accuracy: 0.2451 - loss: 1.8173 - val_accuracy: 0.1886 - val_loss: 1.8696
Epoch 7/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 61s 85ms/step - accuracy: 0.2934 - loss: 1.7813 - val_accuracy: 0.3576 - val_loss: 1.7248
Epoch 8/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 62s 87ms/step - accuracy: 0.3304 - loss: 1.7318 - 

## Matriz de confusion: 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Predicciones
y_pred = np.argmax(model.predict(val_ds), axis=1)

# Etiquetas verdaderas
y_true = []
for _, labels in val_ds.unbatch():
    y_true.append(labels.numpy())
y_true = np.array(y_true)

# Reporte
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Matriz de Confusión')
plt.show()


## Evaluacion del accuaracy

In [51]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/test",
    image_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size
)

test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(buffer_size=AUTOTUNE)

# Evaluar
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.2f}")


Found 7178 files belonging to 7 classes.
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.4862 - loss: 1.3687
Test accuracy: 0.49


## Grafica para ver la accuaracy del modelo:


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Val Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()


## Grafico para ver el desbalance:

In [ ]:
import matplotlib.pyplot as plt

# Conteo por clase
class_counts = {'angry': 3995, 'disgust': 436, 'fear': 4097, 'happy': 7215, 'neutral': 4965, 'sad': 4830, 'surprise': 3171}

# Gráfico
plt.figure(figsize=(10, 6))
plt.bar(class_counts.keys(), class_counts.values(), color='skyblue')
plt.title("Distribución de clases en el dataset de entrenamiento")
plt.xlabel("Emoción")
plt.ylabel("Cantidad de imágenes")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()


## Guardar el modelo

In [ ]:
model.save("emotion_model.h5")
# model.save("emotion_model.keras") <-- Esta forma la recomienda keras ya que el que usamos se ve mas antiguo, pero ocupo confirmar con la documentación